<a href="https://colab.research.google.com/github/FForthmann/Text-Analytics/blob/main/Apple-Product-Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Datenbeschaffung**

Bibliotheken laden

In [ ]:
import requests
import pandas as pd
import spacy
import re
import ast
!pip install matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from collections import Counter
from spacy.lang.de.stop_words import STOP_WORDS as stopwords # DE
from spacy.language import Language
!python -m spacy download de_core_news_sm
!pip install spacy spacy-langdetect
!pip install pyldavis
from spacy_langdetect import LanguageDetector
nlp = spacy.load('de_core_news_sm')
import torch
!pip install textblob-de
import nltk
nltk.download('punkt')
!pip install germansentiment
from germansentiment import SentimentModel



2023-10-08 17:11:07.617303: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 66.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 998.1/998.1 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.7-py3-none-any.whl size=993412 sha256=514c04517f34d0e67000fc431b6e1cb28c795f1b25a8962cd9cbc83be7f51de8
  Stored in directory: /root/.cache/pip/wheels/97/f1/e4/8b73f7a0421b132755956892d29b1e764d3e0857a6e92e32fe
Successfully built langdetect
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 95.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.3 MB/s eta 0:00:00


Daten laden

In [ ]:
# Einfügen API-Schlüssel
#API_SCHLÜSSEL = '18b9fa2ad3e747caa25adcbb0de2bc05' # Lukas
#API_SCHLÜSSEL = '6279420147a54e6bbdfa978580ca2eb0' # Denik
#API_SCHLÜSSEL = '60c356ac8e31492a948de5f25450a51d' # Denik2
#API_SCHLÜSSEL = '694d6c2e280649b39f1ea1dcb6da05ed' # Denik3
#API_SCHLÜSSEL = '022215c8920943eebbac8c759fb5cc10' #Denik 4
#API_SCHLÜSSEL = '07e0bbd957f24f5aa3d1c5f8bb9a42ed'  #Julian

# API-Endpunkt
ENDPUNKT = 'https://newsapi.org/v2/everything'

# Maximaler Zeitraum für die Anfrage (vom 02.09.2023 bis zum 29.09.2023)
start_datum = '2023-09-02'
end_datum = '2023-09-29'

# Suchbegriffe
suchbegriffe = [
    'Apple',
    'iPhone 15',
    'iPhone 15 Plus',
    'iPhone 15 Pro',
    'iPhone 15 Pro Max',
    'Apple Watch Series 9',
    'Apple Watch Ultra 2',
    'AirPods Pro (2. Generation)'
]

# Parameter für die Anfrage
parameter = {
    'language': 'de',  # Sprachparameter für deutsche Artikel
    'sortBy': 'publishedAt',  # Sortiere Artikel nach Veröffentlichungsdatum
    'apiKey': API_SCHLÜSSEL,
    'from': start_datum,
    'to': end_datum,
    'page': 1
}

pageSize = 100
maxArticlesPerKeyword = 2500

# Zählvariable für die Gesamtzahl der Artikel
gesamtzahl_artikel = 0

# Erstellen Sie eine leere Liste, um die Artikel zu speichern
alle_artikel = []

# Durchlaufe die Suchbegriffe und führe separate Anfragen durch
for suchbegriff in suchbegriffe:
    parameter['q'] = suchbegriff  # Setze den aktuellen Suchbegriff in den Parameter

    # Führe die GET-Anfrage durch
    print(parameter)
    antwort = requests.get(ENDPUNKT, params=parameter)

    # Überprüfe, ob die Anfrage erfolgreich war
    print(antwort.status_code)
    daten = antwort.json()
    print("Status:", daten["status"])
    if antwort.status_code == 200:
        anzahl_artikel = daten["totalResults"]  # Anzahl der Artikel für den aktuellen Suchbegriff
        gesamtzahl_artikel += anzahl_artikel
        downloaded_articles = pageSize
        artikel_liste = daten["articles"]
        alle_artikel.extend(artikel_liste)
        result = True
        while downloaded_articles < anzahl_artikel and downloaded_articles < maxArticlesPerKeyword and result:
            parameter['page'] += 1
            print(parameter)
            antwort = requests.get(ENDPUNKT, params=parameter)
            print(antwort.status_code)
            daten = antwort.json()
            print("Status:", daten["status"])
            if antwort.status_code == 200:
              artikel_liste = daten["articles"]
              alle_artikel.extend(artikel_liste)
              downloaded_articles += pageSize
            else:
              result = False # Wenn Fehler, dann Schleife abbrechen
        parameter['page'] = 1 # Parameter auf 1 zurücksetzen, um für den nächsten Suchbegriff wieder auf der ersten Seite zu beginnen

# Erstellen Sie ein Pandas DataFrame aus den gesammelten Artikeln
df = pd.DataFrame(alle_artikel)


# Gesamtzahl der Artikel über alle Suchbegriffe hinweg ausgeben
print("Gesamtzahl der Artikel über alle Suchbegriffe hinweg:", gesamtzahl_artikel)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df = df.drop_duplicates(subset=['title', 'content', 'description'])
df.info()

Data Frame in CSV speichern

In [ ]:
df.to_csv('articles.csv')

# **Datenvalidierung und -aufbereitung**

Vor der weiteren Analyse wird der Datensatz intensiv auf Anomalien
untersucht. Im ersten Schritt wird sich ein Überblick über den Datensatz verschafft.

In [ ]:
full_data = pd.read_csv('articles.csv')

# Display information about the dataset (e.g., number of entries, data type of each column)
print(full_data.info())
print()  # Blank line for better readability

# Display the number of missing values for each column
print(full_data.isnull().sum())
print()  # Blank line for better readability'

# Display the first 5 rows of the dataset
print(full_data.head())
print()  # Blank line for better readability

Der erste Datensatz sieht nach einer Anaomalie aus und wird daher nachfolgend genauer betrachtet. Zur Visualisierung wird eine Tabelle erstellt, welche die Zeilen des ersten Datensatzes aus dem Dataframe wiedergibt

In [ ]:
# Create the new DataFrame for visualization
vis_df = pd.DataFrame({
    '#': range(len(df.columns)),
    'Column': df.columns,
    'Non-Null': df.notnull().sum(),
    'Example': df.iloc[0]
})
vis_df.set_index('#', inplace=True)

# Save the DataFrame as a CSV file
output_path = dateipfad.rsplit('/', 1)[0] + "/table_visualization.csv"
vis_df.to_csv(output_path)

vis_df

Nach der Ansicht des ersten Artikels wird davon ausgegangen, dass der Datensatz auch Artikel beinhaltet die keine relevanten Informationen zu Apple enthält.
Nachfolgend wird daher eine weitere Filterung der Daten durchgeführt. Die Suchbegriffe werden aus der Datenbeschaffung analog eingesetzt.

Außerdem wird eine Bereinigung der Spalten source und publishedAT durchgeführt. Für die Spalte source soll nur den Text bestehen bleiben. Für publishedAt soll das Zeitformat nur das Datum beinhalten.

In [ ]:
# List of search terms
search_terms = [
    'Apple',
    'iPhone 15',
    'iPhone 15 Plus',
    'iPhone 15 Pro',
    'iPhone 15 Pro Max',
    'Apple Watch Series 9',
    'Apple Watch Ultra 2',
    'Apple Watch',
    'AirPods',
    'AirPods Pro \(2nd Generation\)',  # Manuell escaped
    'IOS',
]

# Escape regular expression special characters in search terms
search_terms = [re.escape(term) for term in search_terms]

# Create a regular expression from the search terms
pattern = '|'.join(search_terms)

# Filter DataFrame
filtered_df = full_data[full_data[['title', 'content', 'description']].apply(lambda x: x.str.contains(pattern, case=False, na=False)).any(axis=1)].copy()

# Display information about the dataset
print(filtered_df.info())

##Modifying the 'source' column for better overview
# Function to extract the 'name' value from the string dictionary
def extract_name(source_str):
    try:
        # Wandelt den String in ein Dictionary um
        source_dict = ast.literal_eval(source_str)
        return source_dict.get('name')
    except:
        return source_str

# Apply the function to the 'source' column and edit 'publishedAt'
filtered_df.loc[:, 'source'] = filtered_df['source'].apply(extract_name)
filtered_df.loc[:, 'publishedAt'] = pd.to_datetime(filtered_df['publishedAt'])

Check ob der Datensatz nur noch Artikel enthält, in denen bestimmte Schlüsselwörter vorhanden sind.



In [ ]:
# List of search terms
search_terms = [
    'Apple',
    'iPhone',
    'Apple Watch',
    'AirPods'
]

# Filter the DataFrame based on the search terms
filtered_df_check = filtered_df[filtered_df.apply(lambda row: row.astype(str).str.contains('|'.join(search_terms), case=False).any(), axis=1)]

# Find articles that do not match any of the search terms
non_matching_df = filtered_df[~filtered_df.isin(filtered_df)].dropna()

# Display the number of articles that do not match any of the search terms
print(f"Number of articles without any of the search terms: {len(non_matching_df)}")

# If there are articles that do not match any of the search terms, display some of them
if len(non_matching_df) > 0:
    print("\nSome articles without the search terms:")
    display(non_matching_df.head())

Jetzt wird untersucht, ob im Datensatz tatsächlich nur deutschsprachige Texte enthalten sind.
Es werden die Spalten title, content und description mit einer Spracherkennung untersucht.
Ziel ist es die Top Scores für englische Sprache zu ermitteln und ggf. englischsprachige Texte zu identifizieren.

In [ ]:
# SpaCy-Modell
nlp = spacy.load("en_core_web_sm")

# add Language Detector
@Language.factory("language_detector")
def create_language_detector(nlp, name):
    return LanguageDetector()

nlp.add_pipe("language_detector")

# Function to get Englisch-Score
def get_english_score(text):
    doc = nlp(text)
    if doc._.language['language'] == 'en':
        return doc._.language['score']
    else:
        return 0

# Get the top 10 scores for each column (Title, Description, Content)
for column in ['title', 'description', 'content']:
    # Apply the function to the entire column
    filtered_df[f'{column}_en_score'] = filtered_df[column].astype(str).apply(get_english_score)

    # Get the top 10 scores for this column and only display the selected columns
    top10 = filtered_df.nlargest(10, f'{column}_en_score')[[column, 'source', f'{column}_en_score']]
    print(f"\nTop 10 English Scores for {column}:\n", top10)

Trotz hoher Scores wurde kein englischsprachiger Artikel erkannt. Der teilweise hohe Score ist auf englischen Begriffe wie beispielsweise Apple oder  Watch zurückzuführen. Die Analyse hat jedoch aufgezeigt, dass im Datensatz Werbung enthalten ist. Insbesondere Quellen wie Amazon oder Dealdoktor scheinen Werbecontent zu enthalten. Ziel ist es nun diese Artikel ebenfalls herauszufiltern.

Vorgehen:
1. Offensichtliche Werbung mit bekannten Keywords filtern
2. Quellen untersuchen und manuell überprüften





In [ ]:
print(f"Number of articles before filtering advertisements: {len(filtered_df)}")

# Keywords for Ads
ad_keywords = ["Kaufen Sie jetzt", "Werbung", "Sonderangebot", "Rabatt", "Bestellen Sie", "Anzeige", "Gewinnspiel", "Apple Podcasts", "Deal", "Schnäppchen", "schnaeppchen", "Tarif"]

# filter articles
for keyword in ad_keywords:
    filtered_df = filtered_df[~filtered_df['content'].str.contains(keyword, case=False, na=False)]

print(f"Number of remaining articles after filtering advertisements: {len(filtered_df)}")



Es wurde Werbecontent mit den Schlüsselwörtern identifiziert und im Anschluss entfernt. Nachfolgend werden jetzt die Quellen untersucht.

In [ ]:
# Assuming the column containing the source is named 'source'
unique_sources = filtered_df['source'].drop_duplicates().tolist()

# Assuming the column containing the source is named 'source'
source_counts = filtered_df['source'].value_counts()

# Print the sources and their frequencies
for source, count in source_counts.items():
    print(f"{source}: {count}")

Insbesondere Amazon.de, Mein-deal.com, Dealdoktor.de und Kino-zeit.de sollten genauer untersucht werden.

In [ ]:
# List of sources to check
sources_to_check = ['Amazon.de', 'Mein-deal.com', 'Dealdoktor.de', 'Kino-zeit.de']

# Iterate through the list of sources
for source in sources_to_check:
    source_articles = filtered_df[filtered_df['source'] == source]

    # Display the content and description for articles from this source
    print(f"--------- Articles from {source} ---------")
    for index, row in source_articles.iterrows():
        print(f"Title: {row['title']}\n")
        print(f"Description: {row['description']}\n")
        print(f"Content: {row['content']}\n")
        print("----------------------------------------------------------\n")


Artikel dieser Quellen können entfernt werden. Nun gilt es noch weitere Quellen zu validieren. Insbesondere unter den Quellen mit wenigen Häufigkeiten gibt es einige weitere unerwartete Quellen, wie beispielsweise Süddeutsche Zeitung. Deshalb werden nachfolgend alle Quellen die weniger als 10 mal im Datensatz vorhanden sind manuell überprüft. Der Fokus liegt auf den Spalten Title, Description und Content.

In [ ]:
# Count how often each source occurs
source_counts = filtered_df['source'].value_counts()

# Filter out the sources that appear fewer than 10 times
less_than_10_sources = source_counts[source_counts < 10].index.tolist()

# Filter the DataFrame by these sources
selected_articles = filtered_df[filtered_df['source'].isin(less_than_10_sources)]

# Sort the DataFrame by the source
selected_articles = selected_articles.sort_values(by='source')

# Display the content and description of the selected articles
for index, row in selected_articles.iterrows():
    print(f"Source: {row['source']}")
    print(f"Title: {row['title']}\n")
    print(f"Description: {row['description']}\n")
    print(f"Content: {row['content']}\n")
    print("----------------------------------------------------------\n")


Nach manueller Durchsicht der Quellen wurden weitere Quellen als nicht relevant klassifiziert. Nachfolgend werden diese aus dem Datensatz entfernt und der bereinigte Datensatz wird als neue Datei gespeichert.

In [ ]:
sources_to_remove = [
    'Amazon.de',
    'Mein-deal.com',
    'Dealdoktor.de',
    'Kino-zeit.de',
    'Techstage.de',
    'Süddeutsche Zeitung'
    'Sparbote.de'
    '4Players Portal'
    'Die Zeit'
]

print(f"Number of articles before removing certain sources: {len(filtered_df)}")

# Remove the rows that have one of the above sources as their source
filtered_df = filtered_df[~filtered_df['source'].isin(sources_to_remove)]

print(f"Number of remaining articles after removing certain sources: {len(filtered_df)}")

# Save the new filtered DataFrame as a new CSV file
filtered_df = filtered_df.drop(columns=['Unnamed: 0'])
filtered_df.to_csv('cleaned_and_filtered_articles.csv')

**Zusammenfassung Datenvalidierung und -aufbereitung**

Die API hat wahrscheinlich auch Datensätze erfasst, die bestimmte Wörter wie „Pro“, „Watch“ oder die Zahl 15 in den Texten enthalten. Der ursprünglich abgerufene Datensatz umfasste 1586 Artikel. Nach der Bereinigung von Datensätzen, die nicht auf Apple bezogen sind, sowie von Werbeartikeln, besteht der neue Datensatz nun aus 1175 Kurznachrichtenartikeln. Diese werden im Folgenden weiter bearbeitet.



# **Datennachbearbeitung**

Data Frame aus CSV laden

In [ ]:
full_data = pd.read_csv('cleaned_and_filtered_articles.csv')
full_data.head()

In [ ]:
full_data.info()

In [ ]:
full_data['publishedAt'] = pd.to_datetime(full_data['publishedAt'])

In [ ]:
full_data.info()

In [ ]:
full_data = full_data.dropna() # leere Werte entfernen, da als Float interpretiert

Tokenisierung

In [ ]:
import re
def tokenize_text(texts):
  tokenized_texts = []
  for text in texts:
    tokenized_texts.append(text.split())
  # Sonderzeichen entfernen
  cleaned_tokenized_texts = []
  for text in tokenized_texts:
    cleaned_words = []
    for word in text:
      word = re.sub(r'\b\w*?\.\.\.\w*?\b', '', word) # entfernt Wörter mit "..."
      word = re.sub('[^A-Za-z0-9üÜäÄöÖß]+', ' ', word) # Entfernt alle Sonderzeichen
      word = re.sub(r'\s+', ' ', word) # entfernt mehrfache Leerzeichen
      if word != 'chars' and word.isalnum():
        cleaned_words.append(word)
    cleaned_tokenized_texts.append(cleaned_words)
  return cleaned_tokenized_texts

In [ ]:
def count_words(tokenized_texts):
  word_count = Counter()
  for text in tokenized_texts:
    word_count.update(text)
  return word_count

In [ ]:
full_data['title'] = tokenize_text(full_data['title'])
full_data['description'] = tokenize_text(full_data['description'])
full_data['content'] = tokenize_text(full_data['content'])

In [ ]:
full_data.head()

In [ ]:
count_words(full_data['title']).most_common(10)

In [ ]:
count_words(full_data['description']).most_common(10)

In [ ]:
count_words(full_data['content']).most_common(10)

In [ ]:
print(full_data['title'])

Stopwörter entfernen

In [ ]:
# Remove stopwords
import nltk
nltk.download('stopwords')
stopwords = set(nltk.corpus.stopwords.words('german'))
def remove_stop(tokens):
  cleaned_text = []
  for text in tokens:
    cleaned_text.append([t for t in text if t.lower() not in stopwords])
  return cleaned_text

In [ ]:
full_data['title'] = remove_stop(full_data['title'])
full_data['description'] = remove_stop(full_data['description'])
full_data['content'] = remove_stop(full_data['content'])

In [ ]:
full_data.head()

Lemmatisierung

In [ ]:
nlp = spacy.load('de_core_news_sm')
def lemma(texts):
  lemmatized_texts = []
  for text in texts:
    lemmatized_words = ''
    for word in text:
      lemmatized_words = lemmatized_words + ' ' + nlp(word)[0].lemma_
    lemmatized_texts.append(lemmatized_words)
  return lemmatized_texts

In [ ]:
full_data['title'] = lemma(full_data['title'])
full_data['description'] = lemma(full_data['description'])
full_data['content'] = lemma(full_data['content'])

In [ ]:
full_data.head()

In [ ]:
full_data.to_csv('full_data.csv')

# **Datenexpolartion und -darstellung**

In [ ]:
# Combine text data from the 'title', 'description', and 'content' columns into one column.
text_data = full_data['title'].astype(str) + " " + \
            full_data['description'].astype(str) + " " + \
            full_data['content'].astype(str)

# Join all text rows into one large string.
text = " ".join(text_data)

# Generate the WordCloud.
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

# Visualize the WordCloud.
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

# Sentiment Analyse

**Daten importieren**

In [ ]:
# Definiere den Pfad zur CSV-Datei, die die Daten enthält.
full_data = 'full_data.csv'

# Lese die Daten aus der CSV-Datei in ein Pandas DataFrame namens 'df' ein.
df = pd.read_csv(full_data)

In [ ]:
# Zeige die Form (Anzahl der Zeilen und Spalten) des DataFrames 'df'.
print(df.shape)

In [ ]:
# Definiere eine Liste von Spaltennamen, die aus dem DataFrame 'df' ausgewählt werden sollen.
selected_columns = ['Unnamed: 0', 'source', 'description', 'publishedAt']

# Verwende die Liste von Spaltennamen, um diese Spalten aus dem DataFrame 'df' auszuwählen und ein neues DataFrame namens 'filtered_data' zu erstellen.
filtered_data = df[selected_columns]

# Zeige die ersten paar Zeilen des neuen DataFrames 'filtered_data', um das Ergebnis zu überprüfen.
filtered_data.head()

### <u>**TextBlobDE**</u>

**Textvorschau anhand Zeile 685**

In [ ]:
from textblob_de import TextBlobDE

# Greife auf den Text in Zeile 685 des DataFrames zu und speichere ihn in 'example'.
example = df.at[685, 'description']

# Teile den Text in Wörter auf und wähle die ersten 10 Wörter aus.
words = example.split()[:10]

# Gib die ersten 10 Wörter aus.
print(words)

**POS-Tagging anhand Zeile 685**

In [ ]:
# TextBlobDE-Objekt für den Satz erstellen
blob = TextBlobDE(' '.join(words))

# POS-Tagging für den Satz durchführen
pos_tags = blob.tags  # Gibt eine Liste von Tupeln zurück (Wort, POS-Tag).

# Ausgabe der POS-Tags
print("POS-Tags für die ersten 10 Wörter:", pos_tags)

### <u>**TextBlobDE Sentiment-Analyse**<u>

**Sentiment-Analyse an der Beispielzeile 685**

In [ ]:
# TextBlobDE-Objekt für den Text in 'example' erstellen
example_blob = TextBlobDE(example)

# Sentiment-Analyse für den Text in 'example' durchführen
sentiment = example_blob.sentiment

# Ausgabe des Sentiments
print("Sentiment-Analyse für Zeile 685:")
print("Polarity (Stimmung):", sentiment.polarity)
print("Subjectivity (Subjektivität):", sentiment.subjectivity)

**Durchführung der Sentiment-Analyse für das gesamte DataFrame**

In [ ]:
# Listen zum Speichern der Ergebnisse erstellen
sentiments = []
polarities = []
subjectivities = []

# Durchlaufen jeder Zeile der 'description'-Spalte
for index, row in df.iterrows():
    # Text aus der 'description'-Spalte extrahieren und sicherstellen, dass es ein String ist
    text = str(row['description'])

    # TextBlobDE-Objekt für den Text erstellen
    blob = TextBlobDE(text)

    # Sentiment-Analyse für den Text durchführen
    sentiment = blob.sentiment

    # Ergebnisse speichern
    sentiments.append(sentiment)
    polarities.append(sentiment.polarity)
    subjectivities.append(sentiment.subjectivity)

# Ergebnisse in DataFrame speichern
df['sentiment'] = sentiments
df['polarity'] = polarities
df['subjectivity'] = subjectivities

# Anzeigen der ersten paar Zeilen des DataFrames mit Ergebnissen
print(df[['description', 'sentiment', 'polarity', 'subjectivity']].head())

**Anzeigen der einzelnen Polarity-Werte**

In [ ]:
# Zählen der einzigartigen Polarity-Werte und speichern in einer Serie
polarity_counts = df['polarity'].value_counts()

# Konvertieren der Serie in ein DataFrame und hinzufügen von Spaltenüberschriften
polarity_counts_df = pd.DataFrame({'Polarity': polarity_counts.index, 'Anzahl': polarity_counts.values})

# Sortieren des DataFrame nach der Polarity
polarity_counts_df = polarity_counts_df.sort_values(by='Polarity')

# Ausgabe des DataFrames mit der Anzahl der Polarity-Werte
print(polarity_counts_df)

**Ausgabe der Häufigkeit von Sentiments basierend auf der Polarität**

In [ ]:
# Häufigkeit der Sentiments ausgeben
positive_sentiments = len(df[df['polarity'] > 0])
neutral_sentiments = len(df[df['polarity'] == 0])
negative_sentiments = len(df[df['polarity'] < 0])

print("Anzahl der positiven Sentiments:", positive_sentiments)
print("Anzahl der neutralen Sentiments:", neutral_sentiments)
print("Anzahl der negativen Sentiments:", negative_sentiments)

**Ausgabe drei positive Nachrichten**

In [ ]:
# DataFrame nach der Polarität (Sentiment) sortieren, absteigend (positiv zuerst)
sorted_df = df.sort_values(by='polarity', ascending=False)

# Die drei positivsten Nachrichten auswählen
top_3_positive_news = sorted_df.head(3)

# Die ausgewählten Nachrichten anzeigen
print("Drei positive Nachrichten:")
for index, row in top_3_positive_news.iterrows():
    print("Nachricht:", row['description'])
    print("Polarity (Sentiment):", row['polarity'])
    print()

**Ausgabe drei negative Nachrichten**

In [ ]:
# DataFrame nach der Polarität (Sentiment) sortieren, aufsteigend (negativ zuerst)
sorted_df = df.sort_values(by='polarity', ascending=True)

# Die drei negativsten Nachrichten auswählen
top_3_negative_news = sorted_df.head(3)

# Die ausgewählten Nachrichten anzeigen
print("Drei negative Nachrichten:")
for index, row in top_3_negative_news.iterrows():
    print("Nachricht:", row['description'])
    print("Polarity (Sentiment):", row['polarity'])
    print()

**Durchschnittliche Subjektivität aller Nachrichten**

In [ ]:
# Durchschnitt der Subjektivitäten aus der 'subjectivity'-Spalte des DataFrames berechnen
average_subjectivity = df['subjectivity'].mean()

# Durchschnitt auf zwei Nachkommastellen runden
average_subjectivity = round(average_subjectivity, 2)

# Durchschnittliche Subjektivität aller Nachrichten ausgeben
print("Durchschnittliche Subjektivität aller Nachrichten:", average_subjectivity)

**Durchschnittliche Subjektivität nach Source (absteigend)**

In [ ]:
from tabulate import tabulate

# Gruppieren der Daten nach der Quellenspalte
grouped_data = df.groupby('source')

# Durchschnitt der Subjektivitäten und die Anzahl der Nachrichten pro Quelle berechnen
average_subjectivity_per_source = grouped_data['subjectivity'].mean()
count_per_source = grouped_data.size()

# Durchschnittliche Subjektivität auf 2 Nachkommastellen runden
average_subjectivity_per_source = average_subjectivity_per_source.round(2)

# Erstellung DataFrame, um die Ergebnisse zusammenzufassen
result_df = pd.DataFrame({'Quelle': average_subjectivity_per_source.index, 'Durchschnittliche Subjektivität': average_subjectivity_per_source.values, 'Anzahl Nachrichten': count_per_source.values})

# Ergebnisse nach der durchschnittlichen Subjektivität in absteigender Reihenfolge sortieren
result_df = result_df.sort_values(by='Durchschnittliche Subjektivität', ascending=False)

# Top 5 Quellen auswählen
top_5_sources = result_df.head(5)

# Tabulate verwenden, um die Top 5 Quellen in einer Tabelle zu formatieren
table = tabulate(top_5_sources, headers='keys', tablefmt='pretty', showindex=False)

# Ausgabe der formatierten Tabelle
print(table)

**Durchschnittliches Sentiment (gesamt)**

In [ ]:
# Durchschnittliche Polarität (Sentiment) berechnen
average_sentiment = df['polarity'].mean()

# Durchschnitt auf zwei Nachkommastellen runden
average_sentiment = round(average_sentiment, 2)

# Ausgabe des durchschnittlichen Sentiments
print("Durchschnittliches Sentiment der Nachrichten:", average_sentiment)

**Durchschnittliches Sentiment (vor dem 12.09.2023 17:00)**

In [ ]:
from pytz import timezone

# 'publishedAt'-Spalte in einen datetime-Datentyp mit UTC-Zeitzone konvertieren
df['publishedAt'] = pd.to_datetime(df['publishedAt'], utc=True)

# Schwellenwert für das Datum und die Uhrzeit mit UTC-Zeitzone setzen
threshold_datetime = pd.Timestamp('2023-09-12 17:00:00', tz=timezone('UTC'))

# Nachrichten vor dem Schwellenwert filtern
filtered_df = df[df['publishedAt'] < threshold_datetime]

# Das durchschnittliche Sentiment für die ausgewählten Nachrichten berechnen
average_sentiment = filtered_df['polarity'].mean()

# Durchschnitt auf zwei Nachkommastellen runden
average_sentiment = round(average_sentiment, 2)

# Ausgabe des durchschnittlichen Sentiments
print("Durchschnittliches Sentiment der Nachrichten vor dem 12.09.2023 um 17:00 Uhr:", average_sentiment)

**Durchschnittliches Sentiment (nach dem 12.09.2023 17:00)**

In [ ]:
from pytz import timezone

# 'publishedAt'-Spalte in einen datetime-Datentyp mit UTC-Zeitzone konvertieren
df['publishedAt'] = pd.to_datetime(df['publishedAt'], utc=True)

# Schwellenwert für das Datum und die Uhrzeit auf 12.09.2023 um 17:00:00 mit UTC-Zeitzone setzen
threshold_datetime = pd.Timestamp('2023-09-12 17:00:00', tz=timezone('UTC'))

# Nachrichten nach dem Schwellenwert filtern
filtered_df = df[df['publishedAt'] > threshold_datetime]

# Das durchschnittliche Sentiment für die ausgewählten Nachrichten berechnen
average_sentiment = filtered_df['polarity'].mean()

# Durchschnitt auf zwei Nachkommastellen runden
average_sentiment = round(average_sentiment, 2)

# Ausgabe des durchschnittlichen Sentiments
print("Durchschnittliches Sentiment der Nachrichten nach dem 12.09.2023 um 17:00 Uhr:", average_sentiment)

**Visualisierung des durchschnittlichen Sentiments pro Tag**

In [ ]:
# 'publishedAt'-Spalte in einen datetime-Datentyp mit UTC-Zeitzone konvertieren
df['publishedAt'] = pd.to_datetime(df['publishedAt'], utc=True)

# Daten nach Tag gruppieren und das durchschnittliche Sentiment pro Tag berechnen
average_sentiment_per_day = df.groupby(df['publishedAt'].dt.date)['polarity'].mean().reset_index()

# Diagramm erstellen, um das durchschnittliche Sentiment pro Tag anzuzeigen
plt.figure(figsize=(12, 6))
sns.lineplot(data=average_sentiment_per_day, x='publishedAt', y='polarity', marker='o')
plt.title('Durchschnittliches Sentiment pro Tag')
plt.xlabel('Datum')
plt.ylabel('Durchschnittliche Polarität (Sentiment)')
plt.grid(True)
plt.tight_layout()

# Anzeigen der Grafik
plt.show()

**Ausgabe Tief- und Hochpunkt des Durchschnitts der Polarität**

In [ ]:
# Datum mit dem niedrigsten Durchschnitt der Polarität finden
date_lowest_sentiment = average_sentiment_per_day.loc[average_sentiment_per_day['polarity'].idxmin(), 'publishedAt']
lowest_sentiment = round(average_sentiment_per_day['polarity'].min(), 2)

# Datum mit dem höchsten Durchschnitt der Polarität finden
date_highest_sentiment = average_sentiment_per_day.loc[average_sentiment_per_day['polarity'].idxmax(), 'publishedAt']
highest_sentiment = round(average_sentiment_per_day['polarity'].max(), 2)

# Ausgabe des Tiefpunkts und Hochpunkts
print("Tiefpunkt (niedrigster Durchschnitt der Polarität):")
print("Datum:", date_lowest_sentiment)
print("Durchschnittliche Polarität:", lowest_sentiment)

print("\nHochpunkt (höchster Durchschnitt der Polarität):")
print("Datum:", date_highest_sentiment)
print("Durchschnittliche Polarität:", highest_sentiment)

# Anzeigen der Grafik
plt.show()

**Analyse der durchschnittlichen Polarität nach Quelle**

In [ ]:
# Gruppiere den DataFrame nach der Source und berechne den Durchschnitt der Polarität für jede Source
source_sentiment_avg = df.groupby('source')['polarity'].agg(['mean', 'count']).reset_index()

# Runde den Durchschnitt der Polarität auf zwei Nachkommastellen
source_sentiment_avg['mean'] = source_sentiment_avg['mean'].round(2)

# Sortiere die Quellen absteigend nach der durchschnittlichen Polarität
sorted_sources = source_sentiment_avg.sort_values(by='mean', ascending=False)

# Entferne den Index (Zahlen am Anfang)
sorted_sources.index = sorted_sources['source']
sorted_sources = sorted_sources[['mean', 'count']]

# Ausgabe der besten Quellen (absteigend nach durchschnittlicher Polarität) mit Anzahl der Artikel
print("Die besten Quellen (absteigend nach durchschnittlicher Polarität) und Anzahl der Artikel:")
print(sorted_sources.head())

# Ausgabe der schlechtesten Quellen (absteigend nach durchschnittlicher Polarität) mit Anzahl der Artikel
print("\nDie schlechtesten Quellen (absteigend nach durchschnittlicher Polarität) und Anzahl der Artikel:")
print(sorted_sources.tail())

### <u>**German Sentiment Classification with Bert**<u>

**Sentiment-Analyse an der Beispielzeile 685**

In [ ]:
# Initialisiere das Sentiment-Modell
model = SentimentModel()

# Greife auf den Text in Zeile 685 des DataFrames zu und speichere ihn in 'example'.
example = df.at[685, 'description']

# Führe die Sentiment-Analyse auf dem Beispieltext durch
result = model.predict_sentiment([example])
print(result)

**Sentiment-Analyse an der Beispielzeile 685 inkl. Wahrscheinlichkeiten**

In [ ]:
# Führe die Sentiment-Analyse auf dem Beispieltext durch
classes, probabilities = model.predict_sentiment([example], output_probabilities=True)
print(classes, probabilities)

**Durchführung der Sentiment-Analyse für das gesamte DataFrame**

In [ ]:
# Erstelle leere Listen, um die Ergebnisse zu speichern
sentiments = []
probabilities_list = []

# Iteriere durch alle Zeilen in der Spalte 'description' des DataFrames
for index, row in df.iterrows():
    text = row['description']

    # Überprüfe, ob der Wert vom Typ String ist
    if isinstance(text, str):
        # Führe die Sentiment-Analyse auf dem aktuellen Text durch
        classes, probabilities = model.predict_sentiment([text], output_probabilities=True)

        # Füge die Ergebnisse zur jeweiligen Liste hinzu
        sentiments.append(classes[0])
        probabilities_list.append(probabilities[0])
    else:
        # Falls der Wert kein Text ist, füge Platzhalterwerte hinzu
        sentiments.append(None)
        probabilities_list.append(None)

# Füge die Ergebnisse als neue Spalten zum DataFrame hinzu
df['sentiment'] = sentiments
df['sentiment_probabilities'] = probabilities_list

**Ausgabe der Ergebnisse der Sentiment-Analyse**

In [ ]:
# Iteriere durch den DataFrame und gib die Ergebnisse aus
for index, row in df.iterrows():
    description = row['description']
    sentiment = row['sentiment']
    probabilities = row['sentiment_probabilities']

    print(f"Text: {description}")
    print(f"Sentiment: {sentiment}")
    print(f"Sentiment Wahrscheinlichkeiten: {probabilities}")
    print("-" * 50)  # Trennlinie zwischen den Texten

**Ausgabe der Sentiment-Häufigkeit und Prozentsätze**

In [ ]:
# Filtere die Zeilen aus, bei denen das Sentiment nicht fehlt
filtered_df = df.dropna(subset=['sentiment'])

# Zähle die Anzahl der Vorkommnisse jedes Sentiments
sentiment_counts = filtered_df['sentiment'].value_counts()

# Berechne Prozentsätze für jedes Sentiment
sentiment_percentages = (sentiment_counts / sentiment_counts.sum()) * 100

# Gib die Anzahl der Vorkommnisse und Prozentsätze aus
print("Anzahl der Vorkommnisse jedes Sentiments:")
print(sentiment_counts)
print("\nProzentsätze jedes Sentiments:")
print(sentiment_percentages)

**Ausgabe der drei positivsten Artikel inkl. Wahrscheinlichkeiten**

In [ ]:
# Sortiere den DataFrame nach Sentiment in absteigender Reihenfolge, um die positivsten Artikel zuerst zu erhalten
df_sorted = df.sort_values(by='sentiment', ascending=False)

# Gib die drei positivsten Artikel in ganzen Sätzen aus, inklusive der Wahrscheinlichkeit
for i, row in df_sorted.head(3).iterrows():
    description = row['description']
    sentiment = row['sentiment']
    probabilities = row['sentiment_probabilities']

    print(f"Artikel {i + 1}:")
    print(f"Text: {description}")
    print(f"Sentiment: {sentiment}")
    print(f"Sentiment Wahrscheinlichkeiten: {probabilities}")
    print("-" * 50)  # Trennlinie zwischen den Artikeln

**Ausgabe der drei negativsten Artikel inkl. Wahrscheinlichkeiten**

In [ ]:
# Sortiere den DataFrame nach Sentiment in aufsteigender Reihenfolge, um die negativsten Artikel zuerst zu erhalten
df_sorted = df.sort_values(by='sentiment', ascending=True)

# Gib die drei negativsten Artikel in ganzen Sätzen aus, inklusive der Wahrscheinlichkeit
for i, row in df_sorted.head(3).iterrows():
    description = row['description']
    sentiment = row['sentiment']
    probabilities = row['sentiment_probabilities']

    print(f"Artikel {i + 1}:")
    print(f"Text: {description}")
    print(f"Sentiment: {sentiment}")
    print(f"Sentiment Wahrscheinlichkeiten: {probabilities}")
    print("-" * 50)  # Trennlinie zwischen den Artikeln

**Verteilung der Sentiments (Gesamt)**

In [ ]:
# Gruppiere die Daten nach dem Sentiment und zähle die Anzahl in jeder Gruppe
sentiment_counts = df['sentiment'].value_counts()

# Übersetzung von ursprünglichen Beschreibungen zu neuen Beschreibungen
sentiment_mapping = {
    'negative': 'Negativ',
    'positive': 'Positiv',
    'neutral': 'Neutral'
}

# Mapping anwenden, um die Sentiments umzubenennen
sentiment_counts.index = sentiment_counts.index.map(sentiment_mapping)

# Benutzerdefinierte Farben für die Slices
colors = ['#ff9999', '#66b3ff', '#99ff99']

# Erstelle ein Tortendiagramm
plt.figure(figsize=(8, 6))
plt.pie(sentiment_counts, labels=sentiment_counts.index, autopct='%1.1f%%', startangle=140, colors=colors)
plt.title('Verteilung der Sentiments')

# Hinzufügen einer Legende außerhalb des Diagramms
plt.legend(sentiment_counts.index, title='Sentiments', loc='upper right', bbox_to_anchor=(1.25, 1))

# Zeige das Diagramm an
plt.axis('equal')  # Stellt sicher, dass das Diagramm kreisförmig ist
plt.show()

**Verteilung der Sentiments vor dem 12.09.2023 17 Uhr**

In [ ]:
# Daten vor dem 12.09.2023 um 17 Uhr filtern
filtered_df = df[df['publishedAt'] < '2023-09-12 17:00:00']

# Gruppiere die gefilterten Daten nach dem Sentiment und zähle die Anzahl in jeder Gruppe
sentiment_counts = filtered_df['sentiment'].value_counts()

# Übersetzung von ursprünglichen Beschreibungen zu neuen Beschreibungen
sentiment_mapping = {
    'negative': 'Negativ',
    'positive': 'Positiv',
    'neutral': 'Neutral'
}

# Mapping anwenden, um die Sentiments umzubenennen
sentiment_counts.index = sentiment_counts.index.map(sentiment_mapping)

# Benutzerdefinierte Farben für die Slices
colors = ['#ff9999', '#66b3ff', '#99ff99']

# Erstelle ein Tortendiagramm
plt.figure(figsize=(8, 6))
plt.pie(sentiment_counts, labels=sentiment_counts.index, autopct='%1.1f%%', startangle=140, colors=colors)
plt.title('Verteilung der Sentiments (vor 12.09.2023, 17:00 Uhr)')

# Hinzufügen einer Legende außerhalb des Diagramms
plt.legend(sentiment_counts.index, title='Sentiments', loc='upper right', bbox_to_anchor=(1.25, 1))

# Zeige das Diagramm an
plt.axis('equal')  # Stellt sicher, dass das Diagramm kreisförmig ist
plt.show()

**Verteilung der Sentiments nach dem 12.09.2023 17 Uhr**

In [ ]:
# Daten nach dem 12.09.2023 um 17 Uhr filtern
filtered_df = df[df['publishedAt'] > '2023-09-12 17:00:00']

# Gruppiere die gefilterten Daten nach dem Sentiment und zähle die Anzahl in jeder Gruppe
sentiment_counts = filtered_df['sentiment'].value_counts()

# Übersetzung von ursprünglichen Beschreibungen zu neuen Beschreibungen
sentiment_mapping = {
    'negative': 'Negativ',
    'positive': 'Positiv',
    'neutral': 'Neutral'
}

# Mapping anwenden, um die Sentiments umzubenennen
sentiment_counts.index = sentiment_counts.index.map(sentiment_mapping)

# Benutzerdefinierte Farben für die Slices
colors = ['#ff9999', '#66b3ff', '#99ff99']

# Manuell die Prozentsätze berechnen und auf 100% runden
total = sentiment_counts.sum()
percentages = [(count / total) * 100 for count in sentiment_counts]
rounded_percentages = [round(percentage, 1) for percentage in percentages]

# Erstelle ein Tortendiagramm
plt.figure(figsize=(8, 6))
plt.pie(rounded_percentages, labels=sentiment_counts.index, autopct='%1.1f%%', startangle=140, colors=colors)
plt.title('Verteilung der Sentiments (nach 12.09.2023, 17:00 Uhr)')

# Hinzufügen einer Legende außerhalb des Diagramms
plt.legend(sentiment_counts.index, title='Sentiments', loc='upper right', bbox_to_anchor=(1.25, 1))

# Zeige das Diagramm an
plt.axis('equal')  # Stellt sicher, dass das Diagramm kreisförmig ist
plt.show()

#  Themenmodellierung

TF-IDF berechnen:


In [ ]:
from nltk.corpus import stopwords
# Get german stopwords (append, if necessary)
stopwords = stopwords.words('german')
#stopwords.extend(['sowie','36','de','indeed','workwise']) # Beispiel, ggf. erweitern

In [ ]:
# We need vectorized data
# Let’s calculate the TF-IDF matrix both for the descriptions and for the paragraphs.
tfidf_text_vectorizer = TfidfVectorizer(stop_words=stopwords, min_df=5, max_df=0.7)
tfidf_text_vectors = tfidf_text_vectorizer.fit_transform(full_data['description'])
tfidf_text_vectors.shape

Matrizen berechnen

In [ ]:
nmf_text_model = NMF(n_components=10, random_state=42) # n_components: number of topics
W_text_matrix = nmf_text_model.fit_transform(tfidf_text_vectors)
H_text_matrix = nmf_text_model.components_
W_text_matrix

Themen ermitteln

In [ ]:
def display_topics(model, features, no_top_words=5):
    for topic, words in enumerate(model.components_):
        total = words.sum()
        largest = words.argsort()[::-1] # invert sort order
        print("\nTopic %02d" % topic)
        for i in range(0, no_top_words):
            print("  %s (%2.2f)" % (features[largest[i]], abs(words[largest[i]]*100.0/total)))

display_topics(nmf_text_model, tfidf_text_vectorizer.get_feature_names_out())

LDA

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
# Define the number of topics or components
num_components=8

# Create LDA object
model=LatentDirichletAllocation(n_components=num_components)

# Fit and Transform SVD model on data
lda_matrix = model.fit_transform(tfidf_text_vectors)

# Get Components
lda_components=model.components_

In [ ]:
# Print the topics with their terms
terms = tfidf_text_vectorizer.get_feature_names_out()

for index, component in enumerate(lda_components):
    zipped = zip(terms, component)
    top_terms_key=sorted(zipped, key = lambda t: t[1], reverse=True)[:7]
    top_terms_list=list(dict(top_terms_key).keys())
    print("Topic "+str(index)+": ",top_terms_list)

In [ ]:
print(lda_matrix)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_para_vectorizer = CountVectorizer(stop_words=stopwords, min_df=5, max_df=0.7)
count_para_vectors = count_para_vectorizer.fit_transform(full_data['description'])

from sklearn.decomposition import LatentDirichletAllocation

lda_para_model = LatentDirichletAllocation(n_components = 10, random_state=42)
W_lda_para_matrix = lda_para_model.fit_transform(count_para_vectors)
H_lda_para_matrix = lda_para_model.components_

display_topics(lda_para_model, count_para_vectorizer.get_feature_names_out())

In [ ]:
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
lda_display = pyLDAvis.gensim.prepare(model, lda_matrix, lda_components)